In [ ]:
#mounting drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#installing bnlp_toolkit
!pip install bnlp_toolkit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.7 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171033 sha256=bdc056af5c4154e650935f2e011e6c88aeb8c8afb64eb9e719e9cfa74fadf2aa
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji
  Attempting uninstall: tqdm
    Found existi

In [ ]:

#imporing libraris
#importing all necessari libraries . For tokenization we will use bnlp tokenizer and create our own embedder
from bnlp import NLTKTokenizer
import string
from bnlp import CleanText
import re
import pandas as pd
import numpy as np
import pickle


from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
import matplotlib.font_manager as fm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel
import torch.nn as nn

import torch
import torch.nn as nn

import nltk
nltk.download('punkt')

punkt not found. downloading...


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#reading dataset from drive
data = pd.read_csv('/content/drive/MyDrive/bangladeshnewsData.csv')



In [ ]:
#defined function
#Here we define some functions for our later use

tokenizer = NLTKTokenizer()
clean_text = CleanText(
   fix_unicode=True,
   unicode_norm=True,
   unicode_norm_form="NFKC",
   remove_url=False,
   remove_email=True,
   remove_emoji=True,
   remove_number=True,
   remove_digits=True,
   remove_punct=True,
   replace_with_url="<URL>",
   replace_with_email="<EMAIL>",
   replace_with_number="<NUMBER>",
   replace_with_digit="<DIGIT>",
   replace_with_punct = ""
)

def remove_hyphens(text):
    # Remove hyphens from the text
    cleaned_text = text.replace('\u002D', '')
    return cleaned_text

def remove_unwanted_char(text):
  cleaned_text = text.replace('—','')
  cleaned_text = cleaned_text.replace('<','')
  cleaned_text = cleaned_text.replace('>','')
  cleaned_text = cleaned_text.replace('/','')
  cleaned_text = cleaned_text.replace('...','')

  return cleaned_text

# Chunk text function remains the same
def chunk_text(text, chunk_size=100):
    words = text.split()  # Split the text into words
    return [' '.join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

def remove_parentheses_and_text(lines):
    cleaned_lines = [re.sub(r'\([^)]*\)', '', line) for line in lines]
    return cleaned_lines

def is_bangla(text):
    bangla_pattern = re.compile(r'^[\u0980-\u09FF\s]+$')
    return bool(bangla_pattern.match(text))

def preprocess(text):
  text = clean_text(text)
  text = remove_hyphens(text)
  text = remove_unwanted_char(text)
  text = remove_parentheses_and_text(text)
  #text = is_bangla(text)


  return text

In [ ]:
#As our target is to use sentence embedder
#This functions will separate bangla complete sentences based line ending punctuations

def split_into_sentences(text):
    sentence_endings = ['।', '?', '!']
    sentences = []
    for punctuation in sentence_endings:
        sentences.extend([s.strip() for s in text.split(punctuation) if s.strip()])

    return sentences

def new_split_sentences(text):

    sentences = re.split(r'[।?!]', text)

    return [s.strip() for s in sentences if s.strip()]

In [ ]:
#separate complete sentences and add into a list
text_list = data['Description'].apply(new_split_sentences).to_list()
summary_list = data['Summary'].apply(new_split_sentences).to_list()

In [ ]:
#processing texts
# separate function to process text and summary
def text_preprocess(text_data):
    processed_text = []


    for document in text_data:
        processed_document = []
        for sentence in document:

            temp = clean_text(sentence)
            temp = remove_hyphens(temp)
            temp = remove_unwanted_char(temp)
            processed_document.append(temp)

        t
        processed_text.append(processed_document)

    # Replace '/n' with space (if needed)
    #processed_text = [[[t.replace('/n', " ")] for t in document] for document in processed_text]

    return processed_text

def summary_preprocess(text_data):
    processed_text = []
    for document in text_data:
        processed_document = []
        for sentence in document:

            temp = clean_text(sentence)
            temp = remove_hyphens(temp)
            temp = remove_unwanted_char(temp)
            processed_document.append(temp)

        processed_text.append(processed_document)
        #print(processed_text)
        #break

    # Replace '/n' with space (if needed)
    #processed_text = [[[t.replace('/n', " ")] for t in document] for document in processed_text]

    return processed_text

In [ ]:
#preprocess text and summary
text_data = text_preprocess(text_list)
summary_data = summary_preprocess(summary_list)

In [126]:
# Function to remove non-Bengali words
def remove_non_bengali_words(text_list):
    bengali_pattern = r'[^\u0980-\u09FF\s]'
    cleaned_text = [
        [
            re.sub(bengali_pattern, '', sentence).strip()
            for sentence in sublist
        ]
        for sublist in text_list
    ]
    return cleaned_text

In [128]:
# remove non bengali words from text and summary
text_data = remove_non_bengali_words(text_data)
summary_data = remove_non_bengali_words(summary_data)

In [129]:
#optional
#to check the data
test = text_data[:10]
test

[['গাজীপুরে ছেলে হত্যার বিচার চাইতে গিয়ে প্রতিপক্ষের মারধরে পোশাকশ্রমিক বাবার মৃত্যু হয়েছে বলে অভিযোগ পাওয়া গেছে',
  'আজ বুধবার দুপুরে গাজীপুর সিটি করপোরেশনের বাহাদুরপর এলাকায় এ ঘটনা ঘটে',
  'নিহত ব্যক্তির নাম আসাদুল ইসলাম',
  'তিনি গাইবান্ধার গোবিন্দগঞ্জ উপজেলার পাঁচউলিপুর এলাকার মৃত গোলাম উদ্দিনের ছেলে',
  'নিহতের পরিবার ও পুলিশ সূত্র জানায় আসাদুল ইসলাম গাজীপুর সিটি করপোরেশনের বাহাদুরপুর তুলসীভিটা এলাকার ফিরোজের বাড়িতে সপরিবার ভাড়া থাকেন',
  'সেখান থেকে স্থানীয় একটি পোশাক তৈরি কারখানায় চাকরি করতেন',
  'আগস্ট বেলা টার দিকে তাঁর ছেলে মো নয়ন মিয়াকে  তুরাগ নদে মাছ ধরার কথা বলে বাড়ি থেকে ডেকে আঙ্গুটিয়াচালা এলাকায় নিয়ে যান প্রতিবেশী সেজু  আনন্দ  বাবলু মিয়া  শাহীন হোসেনসহ  কয়েকজন',
  'সেখানে নৌকা থেকে নদে পড়ে নিখোঁজ হয় নয়ন',
  'পরে সন্ধ্যায় তার লাশ ওই নদ থেকে উদ্ধার করেন স্থানীয় ব্যক্তিরা',
  'এ ঘটনায় নৌকা থেকে নদে ফেলে দিয়ে নয়নকে হত্যা করা হয়েছে বলে দাবি করেন আসাদুল ইসলাম',
  'বিষয়টি নিয়ে এলাকায় আসাদুলের ঘনিষ্ঠ সুজন মাহমুদ ও অভিযুক্তদের আত্মীয় আলমগীর হোসেনের স

In [130]:
#function to find out maximum length
def find_max_len(text):
  max_len = 0

  for sub_list in text:

    for line in sub_list:
      l = len(line.split())
      max_len = max(max_len,l)

  return max_len

In [131]:
#find out max len from summary data
max_len = find_max_len(summary_data)

In [132]:
max_len

39

In [133]:
#function to find out depth of input text as it was nested list
def maxi_width(texts):
  max_width = 0
  for sub_list in texts:
    w = len(sub_list)
    max_width = max(w,max_width)


  return max_width


In [134]:
#find out max_width
max_width = maxi_width(text_data)

In [135]:
max_width

178

In [ ]:
#making our dataset
dataset = list(zip(text_data,summary_data))

In [ ]:
#skip or ignore
old_dataset = dataset
dataset = dataset[:700]

In [ ]:
len(dataset)

500

In [136]:


# reduce embedding layer 768 to 100
# as our bert embeddeing layer is 768
reduction_layer = nn.Linear(768, 100)

# function to get sentence embedding using bert model's final hidden state
def get_bangla_sentence_embedding(text, tokenizer, bert_model, max_len):


    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=max_len)

    )
    with torch.no_grad():
        outputs = bert_model(**inputs)

    cls_embedding = outputs.last_hidden_state[:, 0, :]
    reduced_embedding = reduction_layer(cls_embedding)

    return reduced_embedding.squeeze(0)


In [137]:
# function to pad sentence embedding to the max width as input text depth can vary
def pad_sentence_embeddings(sentence_embeddings, max_width)
    num_sentences, embedding_dim = sentence_embeddings.size()

    if num_sentences > max_width:
        padded_embeddings = sentence_embeddings[:max_width, :]
    else:
        padding = torch.zeros((max_width - num_sentences, embedding_dim), device=sentence_embeddings.device)
        padded_embeddings = torch.cat([sentence_embeddings, padding], dim=0)

    return padded_embeddings

In [138]:
# defining our dataset
class BanglaTextSummarizerDataset(Dataset):


  def __init__(self,data,sentence_embedding_function,tokenizer,bert_model,max_len,max_width,pad_sentence_embedding):
    self.data = data
    self.sentence_embedding_function = sentence_embedding_function
    self.tokenizer = tokenizer
    self.bert_model = bert_model
    self.max_len = max_len
    self.width = max_width
    self.pse = pad_sentence_embedding

  def __len__(self):
    return len(self.data)


  def __getitem__(self,idx):

    text,summary = self.data[idx]

    text_embedding = [self.sentence_embedding_function(sentence,self.tokenizer,self.bert_model,self.max_len) for sentence in text]
    text_embedding = torch.stack(text_embedding)
    text_embedding = self.pse(text_embedding,self.width)

    summary_input = " ".join(summary)
    summary_tokens = self.tokenizer(summary_input, return_tensors='pt', truncation=True, padding='max_length', max_length=self.max_len)

    summary_input_ids = summary_tokens['input_ids'].squeeze(0)
    attention_mask = summary_tokens['attention_mask'].squeeze(0)

    return text_embedding, summary_input_ids, attention_mask




In [139]:
# Function to split the dataset into train, eval, and test sets
def split_dataset(data, train_size=0.7, eval_size=0.2, test_size=0.1):
    train_eval_data, test_data = train_test_split(data, test_size=test_size, random_state=42)
    train_data, eval_data = train_test_split(train_eval_data, test_size=eval_size/(train_size+eval_size), random_state=42)
    return train_data, eval_data, test_data

In [140]:
# load tokenizer and bert model
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
bert_model = BertModel.from_pretrained('bert-base-multilingual-cased')

#split dataset
train,val,test = split_dataset(dataset)

In [141]:
#creating train , val and test dataset using custom dataset class

train_dataset = BanglaTextSummarizerDataset(train,get_bangla_sentence_embedding,tokenizer,bert_model,max_len,max_width,pad_sentence_embeddings)
test_dataset = BanglaTextSummarizerDataset(test,get_bangla_sentence_embedding,tokenizer,bert_model,max_len,max_width,pad_sentence_embeddings)
val_dataset = BanglaTextSummarizerDataset(val,get_bangla_sentence_embedding,tokenizer,bert_model,max_len,max_width,pad_sentence_embeddings)

In [142]:
#creating data loader
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
eval_loader = DataLoader(val_dataset, batch_size=8)
test_loader = DataLoader(test_dataset, batch_size=8)

In [ ]:
# optional
for text_embedding,summary_input_ids, attention_mask in train_loader:
    print(f"text Embedding Shape: { text_embedding.shape} ")
    print(f"Summary input Shape:{summary_input_ids.shape}")
    break

text Embedding Shape: torch.Size([20, 178, 100]) 
Summary input Shape:torch.Size([20, 39])


In [143]:

#Modle Architecture
#skip this one
'''
class BanglaTextSummarizerModel(nn.Module):
    def __init__(self, embedding_dim, max_width, vocab_size, num_layers, num_heads, hidden_dim, max_len):
        super(BanglaTextSummarizerModel, self).__init__()
        self.embedding_dim = embedding_dim
        self.max_width = max_width
        self.max_len = max_len

        # Encoder: Transformer Encoder
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=embedding_dim, nhead=num_heads, dim_feedforward=hidden_dim)
        self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)

        # Decoder: Transformer Decoder
        self.decoder_layer = nn.TransformerDecoderLayer(d_model=embedding_dim, nhead=num_heads, dim_feedforward=hidden_dim)
        self.decoder = nn.TransformerDecoder(self.decoder_layer, num_layers=num_layers)

        # Embedding for the target summary (tokens)
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # Linear layer for output vocabulary
        self.fc_out = nn.Linear(embedding_dim, vocab_size)

    def forward(self, text_embedding, input_ids, attention_mask):
        # Encoder forward pass
        encoder_output = self.encoder(text_embedding.permute(1, 0, 2))  # [max_width, batch_size, embedding_dim]

        # Embed input_ids for the decoder
        summary_embeddings = self.embedding(input_ids)  # [batch_size, max_len, embedding_dim]
        summary_embeddings = summary_embeddings.permute(1, 0, 2)  # [max_len, batch_size, embedding_dim]

        # Generate target mask for decoder (causal mask)
        tgt_mask = nn.Transformer.generate_square_subsequent_mask(self.max_len-1).to(input_ids.device)

        # Decoder forward pass
        decoder_output = self.decoder(summary_embeddings, encoder_output, tgt_mask=tgt_mask)  # [max_len, batch_size, embedding_dim]

        # Output layer to get vocabulary logits
        output = self.fc_out(decoder_output)  # [max_len, batch_size, vocab_size]

        return output.permute(1, 0, 2)  # [batch_size, max_len, vocab_size]
        '''

In [144]:
#this to use
# model architecture


class BanglaTextSummarizerModel(nn.Module):
    def __init__(self, embedding_dim, max_width, vocab_size, num_layers, num_heads, hidden_dim,max_len):
        super(BanglaTextSummarizerModel, self).__init__()
        self.embedding_dim = embedding_dim
        self.max_len = max_len
        self.max_width = max_width

        # Encoder: Transformer Encoder
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=embedding_dim, nhead=num_heads, dim_feedforward=hidden_dim)
        self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)

        # Decoder: Transformer Decoder
        self.decoder_layer = nn.TransformerDecoderLayer(d_model=embedding_dim, nhead=num_heads, dim_feedforward=hidden_dim)
        self.decoder = nn.TransformerDecoder(self.decoder_layer, num_layers=num_layers)

        # Embedding for the target summary
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # Linear layer for output vocabulary
        self.fc_out = nn.Linear(embedding_dim, vocab_size)

    def forward(self, text_embeddings, input_ids,attention_mask):

        # Permute text embeddings for the encoder (batch-first to sequence-first)
        encoder_output = self.encoder(text_embeddings.permute(1, 0, 2))  # [max_width, batch_size, embedding_dim]

        # Embed input_ids for the decoder
        summary_embeddings = self.embedding(input_ids)  # [batch_size, max_len, embedding_dim]
        summary_embeddings = summary_embeddings.permute(1, 0, 2)  # [max_len, batch_size, embedding_dim]

        # Generate a causal mask for the decoder
        tgt_seq_len = summary_embeddings.size(0)
        tgt_mask = nn.Transformer.generate_square_subsequent_mask(tgt_seq_len).to(input_ids.device)

        # Decoder forward pass
        decoder_output = self.decoder(summary_embeddings, encoder_output, tgt_mask=tgt_mask)  # [max_len, batch_size, embedding_dim]

        # Output layer to get vocabulary logits
        output = self.fc_out(decoder_output)  # [max_len, batch_size, vocab_size]

        return output.permute(1, 0, 2)  # [batch_size, max_len, vocab_size]


In [145]:
#train and val loop

def train_model(model, train_loader, val_loader, tokenizer, device, num_epochs, lr, patience=3):
    model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

    best_val_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0

        # Training Loop
        for text_embedding, summary_input_ids, attention_mask in train_loader:
            input_ids = summary_input_ids
            text_embedding = text_embedding.to(device)
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)

            # Prepare decoder input and target output
            decoder_input_ids = input_ids[:, :-1]
            decoder_output_ids = input_ids[:, 1:]

            # Forward pass
            optimizer.zero_grad()
            logits = model(text_embedding, decoder_input_ids, attention_mask)

            # Compute loss
            logits = logits.reshape(-1, logits.size(-1))
            decoder_output_ids = decoder_output_ids.reshape(-1)
            loss = criterion(logits, decoder_output_ids)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()

        # Calculate average training loss
        avg_train_loss = total_train_loss / len(train_loader)

        # Validation Loop
        model.eval()
        total_val_loss = 0

        with torch.no_grad():
            for text_embedding, summary_input_ids, attention_mask in eval_loader:

                input_ids = summary_input_ids
                text_embedding = text_embedding.to(device)
                input_ids = input_ids.to(device)
                attention_mask = attention_mask.to(device)

                # Prepare decoder input and target output
                decoder_input_ids = input_ids[:, :-1]
                decoder_output_ids = input_ids[:, 1:]

                # Forward pass
                logits = model(text_embedding, decoder_input_ids, attention_mask)

                # Compute loss
                logits = logits.reshape(-1, logits.size(-1))
                decoder_output_ids = decoder_output_ids.reshape(-1)
                val_loss = criterion(logits, decoder_output_ids)

                total_val_loss += val_loss.item()

        # Calculate average validation loss
        avg_val_loss = total_val_loss / len(val_loader)

        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

        # Check for improvement
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_no_improve = 0
            # Save the best model
            torch.save(model.state_dict(), "best_model.pt")
            print("Validation loss improved. Model saved.")
        else:
            epochs_no_improve += 1
            print(f"No improvement for {epochs_no_improve} epochs.")

        # Early stopping condition
        if epochs_no_improve >= patience:
            print("Early stopping triggered. Training stopped.")
            break

    print("Training complete.")


In [ ]:
#skip
'''
for text_embedding, summary_input_ids, attention_mask in eval_loader:
  print(text_embedding.shape)

  print(type(summary_input_ids))

  break
  '''

torch.Size([8, 178, 100])
<class 'torch.Tensor'>


In [146]:
# Instantiate the model
# Train the model
embedding_dim = 100
max_width = max_width
vocab_size = tokenizer.vocab_size
num_layers = 2
num_heads = 1
hidden_dim = 100
max_len = max_len

model = BanglaTextSummarizerModel(embedding_dim, max_width, vocab_size, num_layers, num_heads, hidden_dim, max_len)


# Train the model with early stopping
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_model(model, train_loader, eval_loader, tokenizer, device, num_epochs=6, lr=0.001, patience=2)

# Load the best model after training
model.load_state_dict(torch.load("best_model.pt"))
print("Best model loaded.")



Epoch [1/6], Train Loss: 10.4848, Val Loss: 8.8414
Validation loss improved. Model saved.
Epoch [2/6], Train Loss: 7.5241, Val Loss: 6.2156
Validation loss improved. Model saved.
Epoch [3/6], Train Loss: 5.7763, Val Loss: 5.4746
Validation loss improved. Model saved.
Epoch [4/6], Train Loss: 5.3666, Val Loss: 5.2986
Validation loss improved. Model saved.
Epoch [5/6], Train Loss: 5.1813, Val Loss: 5.1422
Validation loss improved. Model saved.
Epoch [6/6], Train Loss: 5.0376, Val Loss: 4.9978
Validation loss improved. Model saved.
Training complete.
Best model loaded.


In [ ]:
#You can stop here
#model was not good enough to use in bengali QA

In [ ]:
def input_text_preprocess(text_data):

  processed_text = []

  for sentence in text_data:

      temp = clean_text(sentence)
      temp = remove_hyphens(temp)
      temp = remove_unwanted_char(temp)
      processed_text.append(temp)
    # Replace '/n' with space (if needed)
    #processed_text = [[[t.replace('/n', " ")] for t in document] for document in processed_text]
  return processed_text

In [ ]:
# Function to remove non-Bengali words
def remove_non_bengali_words_1d(text_list):
    bengali_pattern = r'[^\u0980-\u09FF\s]'  # Matches non-Bengali characters
    return [re.sub(bengali_pattern, '', sentence).strip() for sentence in text_list]

In [ ]:
#inferrence text
inp = """অক্টোবর মাসে হামাস নেতা ইয়াহিয়া সিনওয়ারকে হত্যা, লেবাননে হিজবুল্লাহর সঙ্গে যুদ্ধবিরতি ও জানুয়ারিতে নতুন মার্কিন প্রেসিডেন্ট ডোনাল্ড ট্রাম্পের অভিষেক—সব মিলিয়ে, পরিস্থিতি হামাসের অনুকূলে নেই বললেই চলে।

এক ইসরায়েলি কর্মকর্তা বলেন, 'মিসর ও কাতারের মধ্যস্থতাকারীরা বিশ্বাস করেন, ইসরায়েলের প্রতি পক্ষপাতমূলক হলেও হামাস এখন জিম্মি-মুক্তি ও যুদ্ধবিরতির চুক্তি মেনে নিতে পারে।'
      """
#inp = list(inp)
inp_sen = new_split_sentences(inp)
print(inp_sen[0])
processed_inp = input_text_preprocess(inp_sen)
print(processed_inp)
inp_embedding = get_bangla_sentence_embedding(processed_inp, tokenizer, bert_model, max_len)
final_inp =pad_sentence_embeddings(inp_embedding , max_width)
print(final_inp)

অক্টোবর মাসে হামাস নেতা ইয়াহিয়া সিনওয়ারকে হত্যা, লেবাননে হিজবুল্লাহর সঙ্গে যুদ্ধবিরতি ও জানুয়ারিতে নতুন মার্কিন প্রেসিডেন্ট ডোনাল্ড ট্রাম্পের অভিষেক—সব মিলিয়ে, পরিস্থিতি হামাসের অনুকূলে নেই বললেই চলে
['অক্টোবর মাসে হামাস নেতা ইয়াহিয়া সিনওয়ারকে হত্যা লেবাননে হিজবুল্লাহর সঙ্গে যুদ্ধবিরতি ও জানুয়ারিতে নতুন মার্কিন প্রেসিডেন্ট ডোনাল্ড ট্রাম্পের অভিষেকসব মিলিয়ে পরিস্থিতি হামাসের অনুকূলে নেই বললেই চলে', 'এক ইসরায়েলি কর্মকর্তা বলেন মিসর ও কাতারের মধ্যস্থতাকারীরা বিশ্বাস করেন ইসরায়েলের প্রতি পক্ষপাতমূলক হলেও হামাস এখন জিম্মিমুক্তি ও যুদ্ধবিরতির চুক্তি মেনে নিতে পারে', '']
tensor([[ 0.4443,  0.0753,  0.0869,  ...,  0.5878, -0.3117,  0.0410],
        [ 0.6080,  0.1750, -0.1053,  ...,  0.4051, -0.0125, -0.2016],
        [ 0.5691,  0.1789,  0.2714,  ...,  0.6510, -0.4486, -0.2940],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,

In [ ]:
def generate_summary_with_embeddings(model, text_embeddings, tokenizer, max_len):
    # Ensure text_embeddings has the shape [1, sequence_length, embedding_dim]
    if text_embeddings.dim() == 2:  # If embeddings are [embedding_dim]
        text_embeddings = text_embeddings.unsqueeze(0)  # Add batch dimension: [1, sequence_length, embedding_dim]

    # Check the shape after reshaping
    print(f"Reshaped text_embeddings: {text_embeddings.shape}")  # Should be [1, sequence_length, embedding_dim]

    # Step 2: Add [CLS] token at the start of the input sequence
    cls_token_id = tokenizer.cls_token_id
    input_ids = torch.tensor([[cls_token_id]])  # Shape: [1, 1]

    # Step 3: Create an attention mask (1 for valid tokens, 0 for padding tokens)
    attention_mask = torch.ones(text_embeddings.shape[:2])  # Shape: [1, sequence_length]

    # Step 4: Ensure text_embeddings is on the correct device (GPU or CPU)
    text_embeddings = text_embeddings

    # Step 5: Initialize the decoder input with the [CLS] token (start token)
    decoder_input_ids = input_ids  # Start with [CLS] token as the initial sequence

    # Step 6: Generate summary using the model
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        generated_ids = decoder_input_ids  # Start with [CLS] token
        for _ in range(max_len):
            # Forward pass through the model
            output = model(
                text_embeddings=text_embeddings,
                input_ids=generated_ids,
                attention_mask=attention_mask
            )

            # Get logits for the next token in the sequence
            logits = output[:, -1, :]  # Get the logits for the last token in the sequence

            # Apply softmax to get probabilities for each token
            probs = torch.nn.functional.softmax(logits, dim=-1)

            # Get the most likely next token (argmax)
            next_token_id = torch.argmax(probs, dim=-1)

            # Append the next token to the generated sequence
            generated_ids = torch.cat([generated_ids, next_token_id.unsqueeze(0)], dim=1)

            # If the model predicts the [SEP] token, stop generation (or use another stop token)
            if next_token_id.item() == tokenizer.sep_token_id:
                break

    # Step 7: Decode the generated_ids to get the summary text
    summary_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    return summary_text


In [ ]:
generate_summary_with_embeddings(model, final_inp, tokenizer, max_len)

Reshaped text_embeddings: torch.Size([1, 178, 100])


'ম DIGIT জনের মৃত্যুরোনা DIGIT জনের মৃত্যুরোনা DIGIT জনের মৃত্যুরোনা DIGI'

In [148]:
def inference(model, test_loader, tokenizer, device):
    model.to(device)
    model.eval()

    predictions = []

    with torch.no_grad():
        for text_embedding, summary_input_ids, attention_mask in test_loader:
            # Move tensors to the appropriate device
            text_embedding = text_embedding.to(device)
            attention_mask = attention_mask.to(device)

            # Decoder input starts with the <BOS> token (beginning of sequence)
            bos_token_id = tokenizer.cls_token_id
            eos_token_id = tokenizer.sep_token_id
            pad_token_id = tokenizer.pad_token_id

            batch_size = text_embedding.size(0)
            decoder_input_ids = torch.tensor([[bos_token_id] for _ in range(batch_size)], device=device)

            generated_sequences = []

            for _ in range(100):  # Assuming a maximum sequence length of 100
                # Forward pass
                logits = model(text_embedding, decoder_input_ids, attention_mask)

                # Get the next token (argmax from logits)
                next_token_logits = logits[:, -1, :]
                next_token_ids = torch.argmax(next_token_logits, dim=-1).unsqueeze(-1)

                # Append the next token to the sequence
                decoder_input_ids = torch.cat([decoder_input_ids, next_token_ids], dim=1)

                # Stop generating if all sequences in the batch hit the <EOS> token
                if (next_token_ids == eos_token_id).all():
                    break

            # Convert token IDs to text
            for seq in decoder_input_ids:
                decoded_seq = tokenizer.decode(seq, skip_special_tokens=True)
                generated_sequences.append(decoded_seq)

            predictions.extend(generated_sequences)

    return predictions

# Example usage:
# Load the saved model
model.load_state_dict(torch.load("best_model.pt"))

# Perform inference
predictions = inference(model, test_loader, tokenizer, device)

# Display results
for idx, prediction in enumerate(predictions):
    print(f"Prediction {idx + 1}: {prediction}")

Prediction 1: DIGIT জনের মৃত্যু DIGIT জনের মৃত্যুা DIGIT জনের মৃত্যুা DIGIT জনের মৃত্যুা DIGIT জনের মৃত্যুা DIGIT জনের মৃত্যুা DIGIT জনের মৃত্যুা DIGIT জনের মৃত্যুা DIGIT জনের মৃত্যুা DIGIT
Prediction 2: DIGIT জনের মৃত্যু DIGIT জনের মৃত্যুা DIGIT জনের মৃত্যুা DIGIT জনের DIGIT জনের মৃত্যুা DIGIT জনের DIGIT জনের DIGIT জনের DIGIT জনের DIGIT জনের মৃত্যুা DIGIT জনের
Prediction 3: DIGIT জনের মৃত্যু DIGIT জনের মৃত্যুা DIGIT জনের মৃত্যুা DIGIT জনের মৃত্যুা DIGIT জনের মৃত্যুা DIGIT জনের মৃত্যুা DIGIT জনের মৃত্যুা DIGIT জনের মৃত্যুা DIGIT জনের মৃত্যুা DIGIT
Prediction 4: DIGIT জনের মৃত্যু DIGIT জনের মৃত্যুা DIGIT জনের মৃত্যুা DIGIT জনের মৃত্যুা DIGIT জনের মৃত্যুা DIGIT জনের মৃত্যুা DIGIT জনের মৃত্যুা DIGIT জনের মৃত্যুা DIGIT জনের মৃত্যুা DIGIT
Prediction 5: DIGIT জনের মৃত্যু DIGIT জনের মৃত্যুা DIGIT জনের মৃত্যুা DIGIT জনের মৃত্যুা DIGIT জনের মৃত্যুা DIGIT জনের মৃত্যুা DIGIT জনের মৃত্যুা DIGIT জনের মৃত্যুা DIGIT জনের মৃত্যুা DIGIT
Prediction 6: DIGIT জনের মৃত্যু DIGIT জনের মৃত্যুা DIGIT জনের মৃত্

In [ ]:
import torch
import torch.nn as nn

class SimpleEncoderDecoder(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, max_len):
        super(SimpleEncoderDecoder, self).__init__()

        # Encoder does not need an embedding layer, as you're using BERT embeddings
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)

        # Embedding layer for target summary_input_ids (token IDs)
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # Fully connected layer for generating logits
        self.fc = nn.Linear(hidden_dim, vocab_size)

        self.max_len = max_len  # Max length for the output sequence

    def forward(self, text_embedding, summary_input_ids):
        """
        Args:
            text_embedding (torch.Tensor): Input sentence embeddings from BERT (shape: [batch_size, max_width, embedding_dim])
            summary_input_ids (torch.Tensor): Target token IDs (shape: [batch_size, max_len])
        """
        # Pass the text_embedding (BERT embeddings) through the LSTM
        lstm_out, (hidden, cell) = self.rnn(text_embedding)  # Shape: [batch_size, max_width, hidden_dim]

        # Embed the target summary_input_ids for the decoder
        embedded_target = self.embedding(summary_input_ids)  # Shape: [batch_size, max_len, embedding_dim]

        # Initialize the hidden and cell state for the decoder (from encoder's hidden state)
        decoder_input = embedded_target[:, 0, :]  # Start with the first token in the target sequence

        decoder_outputs = []

        # Loop through each time step of the target sequence (max_len)
        for t in range(0, self.max_len):  # Skip the first token because it is already used
            # Pass decoder input through LSTM, using the encoder's hidden state
            decoder_out, (hidden, cell) = self.rnn(decoder_input.unsqueeze(1), (hidden, cell))  # LSTM takes (input, (hidden, cell))

            # Get logits for the current time step (output vocabulary size scores)
            logit = self.fc(decoder_out.squeeze(1))  # Shape: [batch_size, vocab_size]

            # Save the logit to the outputs list (for all time steps)
            decoder_outputs.append(logit)

            # Get the next decoder input as the current token (teacher forcing)
            decoder_input = self.embedding(summary_input_ids[:, t])  # Next token as input

        # Stack the logits for all time steps [batch_size, max_len, vocab_size]
        decoder_outputs = torch.stack(decoder_outputs, dim=1)

        return decoder_outputs


In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader

def train_model(model, train_loader, optimizer, criterion, device, num_epochs=20):
    model.train()

    for epoch in range(num_epochs):
        epoch_loss = 0

        # Loop over batches
        for batch_idx, (text_embedding, summary_input_ids, attention_mask) in enumerate(train_loader):
            text_embedding = text_embedding.to(device)
            summary_input_ids = summary_input_ids.to(device)
            attention_mask = attention_mask.to(device)
            #print(f"summary input ids {summary_input_ids.shape}")
            #summary_input_ids = summary_input_ids[:-1]
            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass through the model
            output = model(text_embedding, summary_input_ids)

                        # Check the shape of output and target to ensure they match
            #print(f"Output shape: {output.shape}")
            #print(f"Target shape: {summary_input_ids.shape}")

            # Flatten the logits and target for loss calculation
            output = output.view(-1, output.size(-1))  # [batch_size * max_len, vocab_size]
            summary_input_ids = summary_input_ids.view(-1)
              # [batch_size * max_len]

            # Compute loss, ignoring padding tokens using attention_mask
            loss = criterion(output, summary_input_ids)

            # Backward pass and optimization step
            loss.backward()
            optimizer.step()

            # Accumulate loss for the current epoch
            epoch_loss += loss.item()

            if batch_idx % 100 == 0:
                print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx}/{len(train_loader)}], Loss: {loss.item():.4f}")

        # Average loss for the epoch
        avg_epoch_loss = epoch_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}] completed. Average Loss: {avg_epoch_loss:.4f}")




In [ ]:
# Initialize model and other components
embedding_dim = 100  # Example embedding dimension (e.g., BERT's embedding size)
hidden_dim = 100
vocab_size = tokenizer.vocab_size  # Example vocab size
max_len = max_len  # Max length of the generated summary

# Create a model instance
model = SimpleEncoderDecoder(embedding_dim, hidden_dim, vocab_size, max_len)

# Move model to GPU (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=0)  # Assuming 0 is padding index in the target summary

# Assuming train_loader is your DataLoader for training data
# train_loader = DataLoader(your_dataset, batch_size=32, shuffle=True)

# Train the model
train_model(model, train_loader, optimizer, criterion, device, num_epochs=20)

Epoch [1/20], Batch [0/2], Loss: 11.7088
Epoch [1/20] completed. Average Loss: 11.7090
Epoch [2/20], Batch [0/2], Loss: 11.6532
Epoch [2/20] completed. Average Loss: 11.6568
Epoch [3/20], Batch [0/2], Loss: 11.6100
Epoch [3/20] completed. Average Loss: 11.6038


KeyboardInterrupt: 